In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.getcwd(), ".."))

from models.sp500.fundamentals import FundamentalStockAnalyzer
import logging

logging.basicConfig(level=logging.CRITICAL)  # or DEBUG for more detail
fas = await FundamentalStockAnalyzer.create()

In [2]:
print("=== Processing data from S&P 500 ===")
recommended_stocks = await fas.get_recommended_stocks(max_concurrent=5)

print("\n=== Top 10 Recommended Stocks (Best Scores) ===")
for i, stock in enumerate(recommended_stocks[:10], 1):
    ticker = stock["ticker"]
    name = stock.get("shortName", "N/A")
    total_score = stock["total_score"]
    annual_change_percent = stock.get("annualChangePercent", "N/A")
    six_month_change_percent = stock.get("sixMonthChangePercent", "N/A")
    three_month_change_percent = stock.get("threeMonthChangePercent", "N/A")
    one_month_change_percent = stock.get("oneMonthChangePercent", "N/A")
    one_week_change_percent = stock.get("oneWeekChangePercent", "N/A")

    # Show some key financial metrics
    operating_cf = stock.get("operatingCashflow", "N/A")
    profit_margins = stock.get("profitMargins", "N/A")
    roa = stock.get("returnOnAssets", "N/A")
    enterprise_value = stock.get("enterpriseValue", "N/A")

    print(f"{i:2d}. {ticker} - {name}")
    print(f"     Total Score: {total_score}")
    print(
        f"     Annual Change: {annual_change_percent}% | 6 Month Change: {six_month_change_percent}% | 3 Month Change: {three_month_change_percent}% | 1 Month Change: {one_month_change_percent}% | 1 Week Change: {one_week_change_percent}%"
    )
    print(
        f"     Operating CF: ${operating_cf:,} | Profit Margin: {profit_margins} | ROA: {roa}"
    )
    print(f"     Enterprise Values: ${enterprise_value:,}")
    print()

=== Processing data from S&P 500 ===

=== Top 10 Recommended Stocks (Best Scores) ===
 1. ADBE - Adobe Inc.
     Total Score: 174
     Annual Change: -32.47% | 6 Month Change: -8.92% | 3 Month Change: -6.42% | 1 Month Change: -2.58% | 1 Week Change: 0.96%
     Operating CF: $9,792,000,000 | Profit Margin: 0.30012 | ROA: 0.18094
     Enterprise Values: $149,900,787,712

 2. ACN - Accenture plc
     Total Score: 138
     Annual Change: -28.27% | 6 Month Change: -21.66% | 3 Month Change: -16.6% | 1 Month Change: -8.65% | 1 Week Change: -0.73%
     Operating CF: $10,949,683,200 | Profit Margin: 0.11607 | ROA: 0.112449996
     Enterprise Values: $147,020,398,592

 3. FI - Fiserv, Inc.
     Total Score: 137
     Annual Change: -25.26% | 6 Month Change: -39.13% | 3 Month Change: -18.75% | 1 Month Change: -4.96% | 1 Week Change: -1.17%
     Operating CF: $6,771,999,744 | Profit Margin: 0.16004999 | ROA: 0.04533
     Enterprise Values: $101,416,992,768

 4. CMG - Chipotle Mexican Grill, Inc.
  